In [1]:
import stellar_sdk
from stellar_sdk import Keypair
import requests
from stellar_sdk import Server, Network, Keypair, TransactionBuilder, Asset
import json
import requests
import argparse
from typing import List, Dict, Union

In [2]:
from stellar_sdk import Keypair

keypair = Keypair.random()
print('address: {}\nsecret:  {}'.format(keypair.public_key, keypair.secret))

address: GAVIXMXFMN7UHI24KSOECM5ZUSRLUSIKZ63HRTQEIADAA5XWVVB6QLFX
secret:  SCIJ5A7GJBAUFKFKMHMWTAGIMOPKYI7UJE3F5RAHBCKJDX5HM6V3RNDY


In [2]:
keypair = Keypair.from_secret('SBHZZ5ROWC47LULDSRWWH7PTDBGAHT2RV7BXHGGZCJCLOLXILNXDBO4F')
public_key = keypair.public_key
can_sign = keypair.can_sign()

print(public_key)
print(can_sign)

GBZWZZ5AWSA5OZXRVUO2NLKDGRPTMYDZRDPIPRP7OAWFHV44XA5USNZ4
True


In [3]:
# Declaring global variable for testnet link.
TESTNET = 'https://horizon-testnet.stellar.org'

# Generate random keypairs and funding them with friendbot.
# Side note: It is common to use a Create Account operation to
# handle the cretion of the distributor account (or any account on mainnet).
# I didn't find it necessary to use for this implementation in order to keep the code
# clean and simple.
def generate_keypair():
    friendbot = 'https://friendbot.stellar.org'
    keypair = Keypair.random()
    requests.get(friendbot, params={'addr': keypair.public_key})
    print('Public key: ', keypair.public_key)
    print('Private key: ', keypair.secret, '\n')
    return keypair

In [4]:
from trader1 import Trader
def get_orders(source: Union[Trader, dict]):
    """Function to get current bids and asks from Horizon
    
    You can either pass a Trader instance, and it will pull the orderbook directly from the server instance,
    or you can pass the orderbook data already.  This is implemented so you can call this function independently with a Trader
    object, or you can stream the orderbook endpoint and pass the data from it directly

    source: Trader or dict - either a Trader object to get orderbook from, or the orderbook data itself
    """
    if type(source) == Trader:
        data = source.server.orderbook(source.base, source.quote).call()
    elif type(source) == dict:
        data = source
    else:
        return None
    
    bids = data['bids']
    asks = data['asks']
    return bids, asks

In [5]:
serv = Server('https://horizon-testnet.stellar.org/')
xlm = Asset.native()
usdc = Asset('USDC', 'GA5ZSEJYB37JRC5AVCIA5MOP4RHTM335X2KGX3IHOJAPP5RE34K4KZVN')
test1 = Trader(serv, "SBHZZ5ROWC47LULDSRWWH7PTDBGAHT2RV7BXHGGZCJCLOLXILNXDBO4F", usdc, xlm)

In [6]:
get_orders(test1)

([], [])

In [32]:
from datetime import datetime, timedelta

RESERVE = None

RESOLUTION_ENUM = {
    '1MIN': 60000,
    '5MIN': 300000,
    '15MIN': 900000,
    '1HOUR': 3600000,
    '1DAY': 86400000,
    '1WEEK': 604800000
}

def get_historical_data(trader: Trader, time_back: timedelta, resolution: str):
    """This method returns a list of historical price data of a SDEX trading pair

    Parameters:
    trader: Trader - Trader object currently being used; needed for trading pair and server instance
    time_back: TimeDelta - amount of time in the past at which the price data should start
    resolution: str - interval of time between each price data point; see RESOLUTION_ENUM

    """
    end = datetime.now()
    end_timestamp = round(end.timestamp()*1000)
    start = end - time_back
    #start_timestamp = round(start.timestamp()*1000)
    start_timestamp = round(start.total_seconds())

    price_data = []
    volume_data = []
    records = trader.server.trade_aggregations(trader.base, trader.quote, RESOLUTION_ENUM[resolution], start_timestamp, end_timestamp, 0)#.cursor('now')
    
    curr = records.call()
    while curr['_embedded']['records'] != []:
        for record in curr['_embedded']['records']:
            price_data.append(float(record['close']))
            volume_data.append(float(record['counter_volume']))

        curr = records.next()

    return price_data, volume_data

In [33]:
get_historical_data(test1,datetime(2022, 5, 30, 16, 28, 56, 125019),'1DAY')

NotFoundError: {
  "type": "https://stellar.org/horizon-errors/not_found",
  "title": "Resource Missing",
  "status": 404,
  "detail": "The resource at the url requested was not found.  This usually occurs for one of two reasons:  The url requested is not valid, or no data in our database could be found with the parameters provided.",
  "extras": {
    "invalid_field": "base_asset",
    "reason": "not found"
  }
}

In [30]:
end = datetime.now()
print(end)
time_back = datetime(2022, 5, 30, 16, 28, 56, 125019)
end_timestamp = round(end.timestamp()*1000)
start = end - time_back
start_timestamp = round(start.total_seconds())
print(start_timestamp)

2022-06-01 16:39:49.120995
173453
